In [1]:
# 分析bao的输出
import os 
import pandas as pd

def analyze_bao_output(file_path):
    with open(file_path)as f:
        lines = f.readlines()
    
    bao_hint = []
    bao_planning_time = []
    bao_exec_time = []
    qid = []
    for line in lines:
        items = line.split(',')
        if items[1].strip() != '2':
            continue
        bao_hint.append(items[0])
        bao_planning_time.append(items[-3])
        bao_exec_time.append(items[-2])
        qid.append(items[-4].split('/')[-1])

    data = {
        'qid': qid, 
        'bao_hint': bao_hint, 
        'bao_planning_time': bao_planning_time, 
        'bao_exec_time': bao_exec_time
    }
    df = pd.DataFrame(data)
    # 提取数字部分并转换为整数（假设格式为 "数字.sql"）
    df['qid_num'] = df['qid'].str.replace('.sql', '')
    
    # 按数字部分从小到大排序
    df = df.sort_values(by='qid_num', ascending=True)
    
    # 删除临时列（可选）
    df = df.drop(columns=['qid_num'])
    
    # df.to_csv('test_bao_tpch-2.csv', index=False)
    return df 

In [3]:
df = analyze_bao_output('test__bao__stats-2.txt')

sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(sum_bao_exec_time)
print(sum_bao_planning_time)

3949.6305509999984
1086.6900509999996


In [3]:
df = analyze_bao_output('test__bao__stats.txt')

sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(sum_bao_exec_time)
print(sum_bao_planning_time)

8183.698072999998
2557.756690999999


In [2]:
df = analyze_bao_output('test__bao__tpch.txt')

sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(sum_bao_exec_time)
print(sum_bao_planning_time)

15517.335328999974
3291.739407000004


In [2]:
df = analyze_bao_output('test__bao__tpch-2.txt')

sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(sum_bao_exec_time)
print(sum_bao_planning_time)

4806.018895000002
14.862372000000008


In [8]:
# base_split_1
df = analyze_bao_output('test__bao__job_bs1.txt')
sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(f'bao_planning: {sum_bao_planning_time}, bao_exec: {sum_bao_exec_time}')


pg_df = analyze_bao_output('test__bao__job_bs1_pg_test.txt')
sum_pg_exec_time = sum(pg_df['bao_exec_time'].astype(float))/1000.0
sum_pg_planning_time = sum(pg_df['bao_planning_time'].astype(float))/1000.0

print(f'pg_planning: {sum_pg_planning_time}, pg_exec: {sum_pg_exec_time}')

gmlr = 1.0
for bao_exec_time, pg_exec_time in zip(df['bao_exec_time'], pg_df['bao_exec_time']):
    gmlr *= (float(bao_exec_time) / float(pg_exec_time))

gmlr = gmlr ** (1.0/len(bao_exec_time))
print(f'gmlr: {gmlr}')

print(df)

bao_planning: 2.55091, bao_exec: 39.611633000000005
pg_planning: 0.3809679999999999, pg_exec: 14.935356000000002
gmlr: 4.724846801540376
        qid                                           bao_hint  \
0   15a.sql  SET enable_mergejoin TO off; SET enable_indexs...   
1   15b.sql  SET enable_mergejoin TO off; SET enable_indexs...   
2   15c.sql                       SET enable_nestloop TO off;    
3   15d.sql                       SET enable_nestloop TO off;    
4   24a.sql                                          (no hint)   
5   24b.sql                      SET enable_mergejoin TO off;    
6   25a.sql  SET enable_mergejoin TO off; SET enable_indexs...   
7   25b.sql                                          (no hint)   
8   25c.sql  SET enable_mergejoin TO off; SET enable_indexs...   
9    2a.sql                                          (no hint)   
10   2b.sql                                          (no hint)   
11   2c.sql                                          (no hint)   
12   

In [9]:
merged_df = pd.merge(df, pg_df, on='qid')
merged_df

,qid,bao_hint_x,bao_planning_time_x,bao_exec_time_x,bao_hint_y,bao_planning_time_y,bao_exec_time_y
0,15a.sql,SET enable_mergejoin TO off; SET enable_indexs...,117.393,342.283,x,14.128,209.969
1,15b.sql,SET enable_mergejoin TO off; SET enable_indexs...,101.706,30.401,x,9.456,23.047
2,15c.sql,SET enable_nestloop TO off;,118.609,1328.738,x,9.611,441.919
3,15d.sql,SET enable_nestloop TO off;,122.842,898.932,x,9.322,619.48
4,24a.sql,(no hint),309.53,239.781,x,58.765,241.827
5,24b.sql,SET enable_mergejoin TO off;,306.288,27.457,x,58.401,23.994
6,25a.sql,SET enable_mergejoin TO off; SET enable_indexs...,99.141,2710.034,x,8.531,1248.18
7,25b.sql,(no hint),120.327,178.006,x,24.962,168.645
8,25c.sql,SET enable_mergejoin TO off; SET enable_indexs...,94.047,7469.056,x,7.945,4684.917
9,2a.sql,(no hint),64.144,434.457,x,0.859,385.257


In [3]:
# leave_ont_out_split_1
df = analyze_bao_output('test__bao__job_l1out1.txt')
sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(f'bao_planning: {sum_bao_planning_time}, bao_exec: {sum_bao_exec_time}')


pg_df = analyze_bao_output('test__bao__job_l1out1_pg.txt')
sum_pg_exec_time = sum(pg_df['bao_exec_time'].astype(float))/1000.0
sum_pg_planning_time = sum(pg_df['bao_planning_time'].astype(float))/1000.0

print(f'pg_planning: {sum_pg_planning_time}, pg_exec: {sum_pg_exec_time}')

gmlr = 1.0
for bao_exec_time, pg_exec_time in zip(df['bao_exec_time'], pg_df['bao_exec_time']):
    gmlr *= (float(bao_exec_time) / float(pg_exec_time))

gmlr = gmlr ** (1.0/len(bao_exec_time))
print(f'gmlr: {gmlr}')

merged_df = pd.merge(df, pg_df, on='qid')
merged_df

bao_planning: 4.7885159999999996, bao_exec: 29.227964
pg_planning: 0.6472490000000001, pg_exec: 38.253988
gmlr: 2.822773360690968


,qid,bao_hint_x,bao_planning_time_x,bao_exec_time_x,bao_hint_y,bao_planning_time_y,bao_exec_time_y
0,10b.sql,SET enable_nestloop TO off;,72.38,1014.855,x,3.208,179.075
1,11b.sql,(no hint),80.065,30.047,x,3.625,25.547
2,12c.sql,SET enable_mergejoin TO off;,104.635,562.803,x,13.741,499.869
3,13b.sql,(no hint),109.303,552.968,x,12.488,466.041
4,14a.sql,(no hint),80.229,229.497,x,3.453,222.566
5,15b.sql,SET enable_mergejoin TO off; SET enable_indexs...,102.11,32.085,x,9.099,22.422
6,16c.sql,SET enable_nestloop TO off;,104.331,2584.432,x,4.722,1440.117
7,17c.sql,SET enable_nestloop TO off;,94.651,1685.92,x,2.734,5499.265
8,18b.sql,(no hint),83.69,160.047,x,3.981,150.272
9,19a.sql,(no hint),135.059,176.813,x,22.147,163.57


In [4]:
# RND_1
df = analyze_bao_output('test__bao__job_RND1.txt')
sum_bao_exec_time = sum(df['bao_exec_time'].astype(float))/1000.0
sum_bao_planning_time = sum(df['bao_planning_time'].astype(float))/1000.0

print(f'bao_planning: {sum_bao_planning_time}, bao_exec: {sum_bao_exec_time}')


pg_df = analyze_bao_output('test__bao__job_rnd1_pg.txt')
sum_pg_exec_time = sum(pg_df['bao_exec_time'].astype(float))/1000.0
sum_pg_planning_time = sum(pg_df['bao_planning_time'].astype(float))/1000.0

print(f'pg_planning: {sum_pg_planning_time}, pg_exec: {sum_pg_exec_time}')

gmlr = 1.0
for bao_exec_time, pg_exec_time in zip(df['bao_exec_time'], pg_df['bao_exec_time']):
    gmlr *= (float(bao_exec_time) / float(pg_exec_time))

gmlr = gmlr ** (1.0/len(bao_exec_time))
print(f'gmlr: {gmlr}')

merged_df = pd.merge(df, pg_df, on='qid')
merged_df

bao_planning: 2.378233, bao_exec: 18.39851
pg_planning: 0.20451599999999998, pg_exec: 26.520885999999997
gmlr: 1.3209219726477173


,qid,bao_hint_x,bao_planning_time_x,bao_exec_time_x,bao_hint_y,bao_planning_time_y,bao_exec_time_y
0,11d.sql,SET enable_mergejoin TO off;,79.485,134.102,x,3.658,124.695
1,15a.sql,(no hint),109.24,229.211,x,14.39,194.735
2,17b.sql,SET enable_nestloop TO off;,98.701,4165.107,x,2.79,5257.165
3,17e.sql,SET enable_nestloop TO off;,89.07,2671.93,x,3.225,9193.088
4,18b.sql,SET enable_mergejoin TO off;,81.396,158.931,x,3.967,137.046
5,1c.sql,SET enable_nestloop TO off;,77.204,246.739,x,0.852,78.106
6,20a.sql,SET enable_nestloop TO off;,126.052,1636.926,x,8.923,1909.342
7,21a.sql,(no hint),101.044,72.83,x,8.281,65.224
8,25c.sql,SET enable_nestloop TO off;,117.001,2088.329,x,8.306,4239.058
9,28b.sql,SET enable_nestloop TO off;,346.571,983.864,x,60.833,1061.266


In [2]:
with open('test__bao__job_imdb.txt')as f:
    lines = f.readlines()

exec_time = 0.0
for line in lines:
    time = line.split(',')[-2]
    exec_time += min(float(time), 5*60*1000)

print(exec_time)
print(exec_time / 1000.0 / 60 / 60)

7029321.552999991
1.952589320277775


In [ ]:
with open('test__bao__tpch.txt')as f:
    lines = f.readlines()

exec_time = 0.0
for line in lines:
    time = line.split(',')[-2]
    exec_time += min(float(time), 5*60*1000)

print(exec_time)
print(exec_time / 1000.0 / 60 / 60)

10974440.298999995
3.0484556386111095


In [1]:
with open('test__bao__job_ext.txt')as f:
    lines = f.readlines()

exec_time = 0.0
for line in lines:
    time = line.split(',')[-2]
    exec_time += min(float(time), 5*60*1000)

print(exec_time)
print(exec_time / 1000.0 / 60 / 60)

9866395.087999998
2.7406653022222214


In [4]:
with open('test__bao__imdb_sub.txt')as f:
    lines = f.readlines()

exec_time = 0.0
planning_time = 0.0
print(len(lines))
for line in lines:
    time = line.split(',')
    exec_time += min(float(time[-2]), 5*60*1000)
    planning_time += min(float(time[-3]), 5*60*1000)

print(exec_time)
print(exec_time / 1000.0 / 60 / 60)
print(planning_time / 1000.0 / 60 / 60)

500
2161424.905000001
0.6003958069444447
0.011081436666666665


In [1]:
with open('test__bao__tpch.txt')as f:
    lines = f.readlines()

exec_time = 0.0
for line in lines:
    time = line.split(',')[-2]
    exec_time += min(float(time), 5*60*1000)

print(exec_time)
print(exec_time / 1000.0 / 60 / 60)

6082453.92
1.6895705333333333


In [4]:
import re

with open('backup4join_order/imdb/test__bao__job_imdb.txt')as f:
    log_text = f.read()

lines = log_text.strip().split('\n')

results = []

for line in lines:
    parts = line.split(',')
    hint = parts[0].strip()
    path_match = re.search(r'/test/([^/]+)\.sql', line)

    if path_match:
        qname = path_match.group(1)
        if hint == '(no hint)' or hint == 'x':
            hint = 'PG'
        results.append(f"{qname}#####{hint}")
    else:
        assert 1

# 保存到文件（可选）
with open('imdb_bao_hint.txt', 'w') as f:
    for item in results:
        f.write(item + '\n')

# 打印结果
for item in results:
    print(item)


c1002#####SET enable_mergejoin TO off;
c1012#####PG
c1065#####SET enable_nestloop TO off;
c107#####SET enable_nestloop TO off;
c1075#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1089#####PG
c1094#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1095#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1103#####SET enable_nestloop TO off;
c1110#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1111#####PG
c1117#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1135#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1146#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1168#####PG
c1173#####SET enable_nestloop TO off;
c12#####PG
c123#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1232#####PG
c1275#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
c1280#####SET enable_nestloop TO off;
c1282#####SET enable_nestloop TO off;
c1288#####PG
c1298#####PG
c1321#####PG
c1330###

In [1]:
import re

with open('backup4join_order/tpch/test__bao__tpch.txt')as f:
    log_text = f.read()

lines = log_text.strip().split('\n')

results = []

for line in lines:
    parts = line.split(',')
    hint = parts[0].strip()
    path_match = re.search(r'/test/([^/]+)\.sql', line)

    if path_match:
        qname = path_match.group(1)
        if hint == '(no hint)' or hint == 'x':
            hint = 'PG'
        results.append(f"{qname}#####{hint}")
    else:
        assert 1

# 保存到文件（可选）
with open('backup4join_order/tpch/tpch_bao_hint.txt', 'w') as f:
    for item in results:
        f.write(item + '\n')

# 打印结果
for item in results:
    print(item)


a1053#####PG
a1063#####PG
a1067#####PG
a1073#####PG
a1086#####PG
a1089#####PG
a1096#####PG
a1111#####PG
a1135#####PG
a1148#####PG
a1158#####PG
a119#####PG
a1221#####PG
a1273#####PG
a1291#####SET enable_nestloop TO off;
a132#####PG
a1343#####PG
a1345#####PG
a1346#####PG
a1359#####PG
a1367#####PG
a1373#####PG
a1400#####PG
a1420#####PG
a143#####PG
a1488#####SET enable_nestloop TO off;
a1510#####PG
a1632#####PG
a1644#####PG
a165#####PG
a1681#####SET enable_nestloop TO off;
a1734#####PG
a1765#####PG
a1774#####SET enable_mergejoin TO off;
a1817#####PG
a1846#####PG
a1857#####PG
a1873#####PG
a1907#####PG
a1938#####PG
a1942#####PG
a1948#####PG
a1952#####PG
a1964#####PG
a2015#####PG
a2032#####SET enable_nestloop TO off;
a2036#####PG
a2137#####SET enable_mergejoin TO off;
a2148#####PG
a2174#####PG
a219#####PG
a2201#####PG
a222#####PG
a2267#####PG
a2309#####PG
a2315#####PG
a2317#####PG
a2335#####PG
a2350#####PG
a238#####PG
a2463#####PG
a2487#####PG
a2502#####PG
a2518#####SET enable_mergejoin TO of

In [1]:
import re

with open('backup4join_order/job-ext/test__bao__job_ext.txt')as f:
    log_text = f.read()

lines = log_text.strip().split('\n')

results = []

for line in lines:
    parts = line.split(',')
    hint = parts[0].strip()
    path_match = re.search(r'/test/([^/]+)\.sql', line)

    if path_match:
        qname = path_match.group(1)
        if hint == '(no hint)' or hint == 'x':
            hint = 'PG'
        results.append(f"{qname}#####{hint}")
    else:
        assert 1

# 保存到文件（可选）
with open('backup4join_order/job-ext/bao_result.txt', 'w') as f:
    for item in results:
        f.write(item + '\n')

# 打印结果
for item in results:
    print(item)


a1006#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1015#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1033#####SET enable_nestloop TO off;
a1052#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1069#####SET enable_nestloop TO off; SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1083#####PG
a1090#####SET enable_nestloop TO off;
a1092#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a11#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1121#####PG
a1128#####SET enable_nestloop TO off;
a1143#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1157#####SET enable_nestloop TO off; SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1176#####SET enable_nestloop TO off;
a1180#####SET enable_mergejoin TO off;
a1184#####SET enable_nestloop TO off;
a1186#####SET enable_nestloop TO off;
a1190#####PG
a1223#####SET enable_mergejoin TO off; SET enable_indexscan TO off;
a1232#####SET enable_nestloo

In [11]:
import re

with open('/home/lgn/source/ai4db_baseline/baseline/Bao/backup4join_order/job-ext/test__bao__job_ext.txt')as f:
    log_text = f.read()

lines = log_text.strip().split('\n')

results = []

for line in lines:
    parts = line.split(',')
    plan_time = float(parts[-3].strip())
    if plan_time < 3000:
        results.append(plan_time)

print(len(results))
print(len(lines))
print(sum(results) / 1000.0)
print(sum(results) * len(lines) / len(results) / 1000.0)


496
496
36.86462600000001
36.86462600000001
